# Report with IA

**Descrição**
* O objetivo do projeto foi criar um modelo de regressão com séries temporais, treinado com o intuito de prever o faturamento diário de uma cafeteria ao longo de um mês;
* O objetivo do deploy é criar um report com IA contendo as previsões do modelo. Dessa forma, o cliente poderia acompanhar bem as métricas e variações apresentadas de forma automatizada

**Etapas:**

* Prompt para a geração do relatório
* Configurações do relatório

# 1. Bibliotecas e importação dos dados

In [1]:
# Gerais

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
from dotenv import load_dotenv
import os
import re

# Geração de PDF com o ReportLab

# Define o tamanho da página:
from reportlab.lib.pagesizes import letter
# Define os estilos de texto:
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
# Para a criação de documentos e elementos como parágrafos, tabelas e imagens:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
# Para unidades de medida:
from reportlab.lib.units import inch
# Para usar cores:
from reportlab.lib import colors

# Bibliotecas para trabalhar com markdown e conversão (ex: HTML)

import markdown2

In [2]:
# Importação

df = pd.read_csv('data/report_data.csv',index_col=0)

In [3]:
df.head()

,ano-mes-dia,faturamento
0,2023-06-01,3501.3345
1,2023-06-02,3687.8699
2,2023-06-03,3512.5303
3,2023-06-04,3627.7790
4,2023-06-05,3570.6147


In [4]:
# Parâmetros avaliados na seção de resumos financeiros

total_faturamento = np.round(df['faturamento'].sum(),2)
describe = np.round(df.describe(),2).reset_index()
describe.rename(columns={'index': 'describes'},inplace=True)
nova_linha = pd.DataFrame([{'describes': 'total', 'faturamento': total_faturamento}])
describe = pd.concat([describe, nova_linha], ignore_index=True)
describe = describe[~describe['describes'].isin(['count','25%', '50%', '75%'])].reset_index(drop=True)
describe

,describes,faturamento
0,mean,4075.03
1,std,374.38
2,min,3501.33
3,max,4664.56
4,total,122250.95


# 2. Acessando a API da OpenAI

In [5]:
# Acessando API

load_dotenv()
secret_key = os.getenv("OPEN_API_KEY")

In [6]:
# Instanciando

client = openai.OpenAI(api_key=secret_key)

# 3. Prompt para a geração do relatório com o GPT-4

In [7]:
# Passando o dataframe com as previsões para texto

data_text = df.to_string(index=False)
data_text

'ano-mes-dia  faturamento\n 2023-06-01    3501.3345\n 2023-06-02    3687.8699\n 2023-06-03    3512.5303\n 2023-06-04    3627.7790\n 2023-06-05    3570.6147\n 2023-06-06    3772.3198\n 2023-06-07    3763.6150\n 2023-06-08    3899.9277\n 2023-06-09    3988.8083\n 2023-06-10    4267.1177\n 2023-06-11    4279.8730\n 2023-06-12    4398.9785\n 2023-06-13    4417.1170\n 2023-06-14    4552.8600\n 2023-06-15    4664.5620\n 2023-06-16    4612.7760\n 2023-06-17    4499.0537\n 2023-06-18    4401.6606\n 2023-06-19    4496.7974\n 2023-06-20    4496.4673\n 2023-06-21    4352.6760\n 2023-06-22    4234.0557\n 2023-06-23    4042.9932\n 2023-06-24    4064.5315\n 2023-06-25    4172.6597\n 2023-06-26    4100.8960\n 2023-06-27    4026.1042\n 2023-06-28    3753.1160\n 2023-06-29    3528.1300\n 2023-06-30    3563.7275'

In [8]:
# Passando o dataframe com as estatísticas descritivas para texto

data_describe_text = describe.to_string()
data_describe_text

'  describes  faturamento\n0      mean      4075.03\n1       std       374.38\n2       min      3501.33\n3       max      4664.56\n4     total    122250.95'

In [9]:
# Prompt gerado para o GPT-4

prompt = f"""
Você é um analista de dados eficiente e preciso. Baseado nas seguintes previsões de faturamento diário de uma cafeteria,
gere um relatório curto e conciso sumarizando as principais tendências, pontos notáveis e potenciais insights:

{data_text}

{data_describe_text}

Seu relatório deve conter as seções:
1- Tendência geral: uma análise curta da tendência geral.
2- Visão de negócio: possíveis implicações de negócio e estratégias que podem ser adotadas.
3- Resumo financeiro: um resumo financeiro sucinto, destacando o faturamento total do mês, a média diária, e as maiores oscilações (maior e menor faturamento). Separar em tópicos dentro da seção 4. (Usar * para o markdown converter)
Use esses dados como referência para comentar qualquer dado numérico no relatório.
4- Considerações finais: um resumo dos principais pontos e conclusões da análise.

Configurações do texto:
1- Entregue uma análise concisa, porém informativa.
2- A análise deve ser devolvida em markdown.
3- Deixar apenas as iniciais de uma frase maiúscula em todo o texto, ex:"Tendência geral" e não "Tendência Geral".
4- Não pular tantas linhas "\n\n" dentro de uma mesma seção, apenas entre diferentes seções.
5- Para fins de clareza, organize o relatório com as seções bem definidas, com títulos destacados **(em negrito)**.
6- Nos bullet points do resumo financeiro, destaque os títulos em negrito.
7- O restante do conteúdo pode seguir em texto simples.
8- Use o formato Markdown para listas com marcadores (como “*” para os itens de lista) e subtítulos em negrito, para facilitar a leitura e organização do conteúdo.

"""

In [10]:
completion = client.chat.completions.create(
  model="gpt-4o-mini", store=True, temperature=0, max_tokens=2000,
  messages=[{"role": "system", "content": "Você é um analista de dados eficiente."},
              {"role": "user", "content": prompt}]
)

In [11]:
# Salvando a parte de interesse do output gerado pelo modelo

html_report = completion.to_dict()['choices'][0]['message']['content']
html_report

'**Tendência geral**  \nA análise do faturamento diário da cafeteria durante o mês de junho de 2023 revela uma tendência de crescimento consistente, especialmente nas duas primeiras semanas. O faturamento aumentou de R$ 3.501,33 no dia 1º para um pico de R$ 4.664,56 no dia 15, antes de apresentar uma leve queda nas semanas seguintes. A variação diária mostra um padrão de flutuação, mas a tendência geral é de alta.\n\n**Visão de negócio**  \nCom base nas tendências observadas, a cafeteria pode considerar as seguintes estratégias:  \n* Implementar promoções ou eventos especiais durante os dias de maior movimento para maximizar o faturamento.  \n* Analisar os produtos mais vendidos durante os dias de pico e ajustar o cardápio ou as ofertas para atender à demanda.  \n* Avaliar a possibilidade de expandir o horário de funcionamento ou aumentar a equipe durante os períodos de maior movimento para melhorar o atendimento e a experiência do cliente.\n\n**Resumo financeiro**  \n* **Faturamento t

In [12]:
# html_report = '**Relatório de Análise de Faturamento - Cafeteria (Junho de 2023)**\n\n**Análise da tendência geral**  \nO faturamento diário da cafeteria apresentou uma tendência de crescimento ao longo do mês de junho de 2023, com um aumento significativo nas receitas, especialmente na segunda quinzena do mês. O faturamento começou em torno de 3.500 e alcançou picos superiores a 4.400, indicando um aumento na demanda.\n\n**Identificação de possíveis oscilações**  \nEmbora a tendência geral seja de crescimento, foram observadas oscilações nos dias 28 e 29, onde o faturamento caiu para 3.753 e 3.528, respectivamente. Essas oscilações podem ser atribuídas a fatores como condições climáticas ou eventos locais que impactaram o fluxo de clientes.\n\n**Possíveis implicações de negócio**  \nO aumento no faturamento sugere uma oportunidade para a cafeteria expandir suas operações, talvez considerando a introdução de novos produtos ou promoções. No entanto, as oscilações indicam a necessidade de monitorar fatores externos que possam afetar a receita, como feriados ou eventos locais.\n\n**Resumo financeiro**  \n* Faturamento total do mês: 123.000,00  \n* Média diária: 4.100,00  \n* Maiores oscilações:  \n  * Maior faturamento: 4.417,12 (13 de junho)  \n  * Menor faturamento: 3.501,33 (1 de junho)  \n  * Oscilações significativas: 4.664,56 (15 de junho) para 4.401,66 (18 de junho)\n\n**Oscilação e picos de faturamento**  \nO pico de faturamento observado no dia 15 de junho (4.664,56) pode estar relacionado a eventos locais ou promoções específicas. Não há feriados nacionais nos EUA durante junho que possam justificar um aumento significativo, mas eventos comunitários ou promoções de verão podem ter atraído mais clientes.\n\n**Considerações finais**  \nO mês de junho de 2023 foi positivo para a cafeteria, com um crescimento notável no faturamento. As oscilações observadas devem ser monitoradas para entender melhor os fatores que influenciam a demanda. A análise sugere que a cafeteria está em uma trajetória de crescimento, mas deve estar atenta a fatores externos que possam impactar suas receitas.'

In [13]:
html_report

'**Tendência geral**  \nA análise do faturamento diário da cafeteria durante o mês de junho de 2023 revela uma tendência de crescimento consistente, especialmente nas duas primeiras semanas. O faturamento aumentou de R$ 3.501,33 no dia 1º para um pico de R$ 4.664,56 no dia 15, antes de apresentar uma leve queda nas semanas seguintes. A variação diária mostra um padrão de flutuação, mas a tendência geral é de alta.\n\n**Visão de negócio**  \nCom base nas tendências observadas, a cafeteria pode considerar as seguintes estratégias:  \n* Implementar promoções ou eventos especiais durante os dias de maior movimento para maximizar o faturamento.  \n* Analisar os produtos mais vendidos durante os dias de pico e ajustar o cardápio ou as ofertas para atender à demanda.  \n* Avaliar a possibilidade de expandir o horário de funcionamento ou aumentar a equipe durante os períodos de maior movimento para melhorar o atendimento e a experiência do cliente.\n\n**Resumo financeiro**  \n* **Faturamento t

# 4. Configurações do relatório

### 4.1 Configurações do gráfico

In [14]:
# Definição do caminho do gráfico
graph_path = "data/grafico.png"

# Garantindo que a coluna 'ano-mes-dia' seja interpretada como datas
df['ano-mes-dia'] = pd.to_datetime(df['ano-mes-dia'])

# Calculando a média do faturamento
media_faturamento = df['faturamento'].mean()

# Criando o gráfico de tendência de faturamento
plt.figure(figsize=(8, 4), dpi=300)
plt.plot(df['ano-mes-dia'], df['faturamento'], marker='o', linestyle='-', color='#D2691E')

# Adicionando a linha da média
plt.axhline(y=media_faturamento, color='#FFA500', linestyle='--', label=f'Média: R$ {media_faturamento:.2f}')

# Título e rótulos
plt.title("Análise temporal do faturamento", fontsize=12)
plt.xlabel("Dias (Junho/2023)", fontsize=10)
plt.ylabel("Faturamento (R$)", fontsize=10)
plt.legend() # Adicionando a legenda

# Ajustando os rótulos do eixo X para mostrar apenas dia e mês
plt.xticks(
    df['ano-mes-dia'],  # Usando as datas reais
    df['ano-mes-dia'].dt.strftime('%d'),  # Formatando para exibir apenas dia e mês
    ha='right', # Alinhando melhor as legendas
    fontsize=8
)

# Ajustando os limites dos eixos para garantir que o gráfico não fique apertado
plt.xlim(min(df['ano-mes-dia']) - pd.Timedelta(days=1), max(df['ano-mes-dia']) + pd.Timedelta(days=1)) 
plt.ylim(min(df['faturamento']) - 100, max(df['faturamento']) + 100)

# Ajuste automático do layout para evitar sobreposição
plt.tight_layout()

plt.savefig(graph_path)
plt.close()
#plt.show()

### 4.2 Configurações do pdf

In [15]:
# Criando o relatório em PDF
html_report = markdown2.markdown(html_report)

pdf_filename = "sales_report.pdf"
doc = SimpleDocTemplate(
    pdf_filename,
    pagesize=letter,
    leftMargin=40, rightMargin=40, topMargin=20, bottomMargin=20
)
styles = getSampleStyleSheet()
story = []

# Estilos personalizados
styles["Title"].fontSize = 18
styles["Title"].spaceAfter = 12
styles["Title"].textColor = "#8B4513"

styles["Heading1"].fontSize = 14
styles["Heading1"].spaceBefore = 12
styles["Heading1"].spaceAfter = 6
styles["Heading1"].textColor = colors.black

styles["BodyText"].fontSize = 12
styles["BodyText"].leading = 14
styles["BodyText"].spaceAfter = 6
styles["BodyText"].alignment = 4  # Justificado apenas no corpo do texto

# Definindo estilos para bullets aninhados
styles.add(ParagraphStyle(
    name="BulletLevel1",
    parent=styles["BodyText"],
    leftIndent=12,
    bulletIndent=0,
))
styles.add(ParagraphStyle(
    name="BulletLevel2",
    parent=styles["BodyText"],
    leftIndent=24,
    bulletIndent=12,
))

# Adicionando o título do relatório
story.append(Paragraph("<b>Relatório de previsão de faturamento</b>", styles["Title"]))
story.append(Spacer(1, 0.2 * inch))

# Configurações do parágrafo com bullets aninhados
for raw_line in html_report.split("\n"):
    text = raw_line.strip()
    if not text:
        continue

    # Calculando indentação para bullets de segundo nível
    indent = len(raw_line) - len(raw_line.lstrip(" "))
    if text.startswith("* "):
        clean = text[2:].strip()
        lvl = "BulletLevel2" if indent >= 2 else "BulletLevel1"
        story.append(Paragraph(clean, styles[lvl], bulletText="•"))
    else:
        story.append(Paragraph(text, styles["BodyText"]))

    story.append(Spacer(1, 0.2 * inch))

### 4.3 Configurações da tabela

In [16]:
# Formatando a coluna 'ano-mes-dia' para exibir apenas dia/mês/ano
df['ano-mes-dia'] = df['ano-mes-dia'].dt.strftime('%d/%m/%Y')

# Arredondando os valores de faturamento para 2 casas decimais
df['faturamento'] = df['faturamento'].round(2)

# Criando a tabela com os dados ajustados
data_table = [["Data", "Previsão (R$)"]] + df[['ano-mes-dia', 'faturamento']].values.tolist()

# Definindo a tabela
table = Table(data_table, colWidths=[150, 150])

# Aplicando o estilo da tabela
table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), '#8B4513'),  # Cor de fundo para o cabeçalho
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),  # Cor do texto do cabeçalho
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Alinhamento do texto na tabela
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Fonte do cabeçalho
    ('BOTTOMPADDING', (0, 0), (-1, 0), 8),  # Padding inferior para o cabeçalho
    ('BACKGROUND', (0, 1), (-1, -1), '#F5F5E4'),  # Cor de fundo para as células da tabela
    ('GRID', (0, 0), (-1, -1), 1, '#D3D3D3'),  # Estilo de grade
]))

# Adicionando a tabela ao relatório
story.append(Spacer(1, 0.3 * inch))
story.append(Paragraph("Previsões de faturamento", styles["Heading1"]))
story.append(Spacer(1, 0.1 * inch))
story.append(table)

# Adicionando um Spacer após a tabela para o gráfico não ficar tão próximo
story.append(Spacer(1, 1 * inch))

# Adicionando o gráfico
story.append(Paragraph("Tendência de faturamento", styles["Heading1"]))
story.append(Spacer(1, 0.2 * inch))
story.append(Image(graph_path, width=600, height=300))
story.append(Spacer(1, 0.5 * inch))

In [17]:
# Gerando o PDF
doc.build(story)